In [3]:
#%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install azureml-train-automl-runtime
#%pip install --upgrade azureml-sdk[notebooks,automl] -q
%pip install azureml-sdk
%pip install azureml-widgets

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 0, 14, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.7/420.7 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 118.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 KB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 93.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 KB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.6/249.6 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 KB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 98.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 KB 66.3 MB/s eta 0:00

In [2]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.from_config('Users/odl_user_269054/config.json')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 19, Finished, Available, Finished)

Workspace name: quick-starts-ws-269054
Azure region: eastus2
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-269054


# **Set up cluster**

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException 

cluster_name = "project01-azureml"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target:', cluster_name)
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=25)

# get a detailed status for the current cluster using get_status()
print(compute_target.get_status().serialize())

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 20, Finished, Available, Finished)

Found existing compute target: project01-azureml
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 2, 'runningNodeCount': 2, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-10-24T14:58:57.624000+00:00', 'errors': None, 'creationTime': '2024-10-24T14:41:41.160672+00:00', 'modifiedTime': '2024-10-24T14:41:51.037069+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'Standard_D2_V2'}


### **Sep up hyperparameters**

In [4]:
from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn

from azureml.train.hyperdrive import PrimaryMetricGoal, BanditPolicy, RandomParameterSampling, HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--max_iter': choice(50, 75, 100),
        '--C': uniform(0.1, 10)
    }
)
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='Users/odl_user_269054/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', script='Users/odl_user_269054/train.py', compute_target=compute_target, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 21, Finished, Available, Finished)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(hyperdrive_config)
run.wait_for_completion(show_output=True)

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 22, Finished, Available, Finished)

RunId: HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104
Web View: https://ml.azure.com/runs/HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-269054/workspaces/quick-starts-ws-269054&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-10-24T15:00:23.5220756Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-10-24T15:00:23.7744240Z][SCHEDULER][INFO]Scheduling job, id='HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_0' 
[2024-10-24T15:00:23.8557025Z][SCHEDULER][INFO]Scheduling job, id='HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_2' 
[2024-10-24T15:00:23.8216311Z][SCHEDULER][INFO]Scheduling job, id='HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_1' 
[2024-10-24T15:00:23.8951779Z][SCHEDULER][INFO]Scheduling job, id='HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_3' 
[2024-10-24T15:00:24.3017121Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_0' 
[2024-10-24T15:0

{'runId': 'HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104',
 'target': 'project01-azureml',
 'status': 'Completed',
 'startTimeUtc': '2024-10-24T15:00:22.61677Z',
 'endTimeUtc': '2024-10-24T15:08:58.986988Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9555aa11-c5c8-478a-bc50-1c2db9f87fc9',
  'user_agent': 'python/3.10.6 (Linux-4.15.0-1178-azure-x86_64-with-glibc2.27) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.58.0',
  'best_child_run_id': 'HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_6',
  'score': '0.9109256449165404',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_7de82134-bb7c-42c3-a0e1-f0294a1bb104_6'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-

In [6]:
# Check the best run metrics
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Best configs: ",best_run_metrics)

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 23, Finished, Available, Finished)

Best configs:  {'Regularization Strength:': 0.2666881067529016, 'Max iterations:': 50, 'Accuracy': 0.9109256449165403}


In [7]:
import joblib
# Get your best run and save the model from that run.
best_model = [x for x in best_run.get_file_names() if '.joblib' in x][0]
best_run.download_file(name=best_model, output_file_path='./training')
print('Model saved')

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 24, Finished, Available, Finished)

Model saved


# **AutoML**

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 25, Finished, Available, Finished)

In [20]:
# Check the current working directory to import the train.py
import os

print("Current working directory:", os.getcwd())
print("Files in current directory:", os.listdir('.'))

StatementMeta(218b131a-7e29-46ae-b1ac-b2f9a1969206, 1, 35, Finished, Available, Finished)

Current working directory: /synfs/notebook/1/aml_notebook_mount
Files in current directory: ['Users', 'azureml-logs', 'training', 'logs', 'outputs']


In [21]:
# Add train.py into sys.path 
#import sys
#sys.path.append(os.path.join(os.getcwd(), 'Users', 'odl_user_269054'))

StatementMeta(218b131a-7e29-46ae-b1ac-b2f9a1969206, 1, 36, Finished, Available, Finished)

In [9]:
def clean_data(data):
    # Dict for mapping the months and weekdays
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Drop na, create new columns for onehot encode and drop the original 'job' column
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    # Change the string into binary form is 1 == yes, 0 == no for columns
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    # Drop na, create new columns for onehot encode and drop the original 'contact' column
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    # Drop na, create new columns for onehot encode and drop the original 'education' column
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    # Replace strings with number as map define above
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    # Change the string into binary form is 1 == yes, 0 == no for columns
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    # Taken steps seem to be independence from each other, but arrange inefficient
    # But wanna make sure the code work and have limit of time this account so not gonna change a thing
    # Just comments for future reference
    return x_df, y_df

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 26, Finished, Available, Finished)

In [10]:
#from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

train_df = pd.concat([x_train,y_train], axis=1)

train_df.to_csv('train.csv', index=False)

# Setup data store
datastore = ws.get_default_datastore()

# Register the dataset from pd df
dataset = Dataset.Tabular.register_pandas_dataframe(dataframe=train_df, target=(datastore, 'train.csv'),  name='bankmarketing_train_automl')

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 27, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'ef1a8181-4872-4771-b5e7-f4c4dddb2d02'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'ef1a8181-4872-4771-b5e7-f4c4dddb2d02', 'run_id': 'ef1a8181-4872-4771-b5e7-f4c4dddb2d02'}
Validating arguments.
Arguments validated.
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header

In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_hours=0.3,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',  # Ensure this is the correct label column name
    n_cross_validations=5,
    compute_target=compute_target
)

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 37, Finished, Available, Finished)

In [21]:
# Submit your automl run
experiment = Experiment(ws, "automl_test_experiment")
automl_run = exp.submit(config=automl_config, show_output=True)

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 38, Finished, Available, Finished)

Submitting remote run.
No run_configuration provided, running on project01-azureml with default configuration
Running on remote compute: project01-azureml


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f82c26ba-adb3-439c-b8dd-769b1eea6e64,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [22]:
# Retrieve and save your best automl model.
best, model = automl_run.get_output()
joblib.dump(model, './training/bestAutoML.joblib')

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 39, Finished, Available, Finished)

[16:31:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[16:31:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[16:31:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

['./training/bestAutoML.joblib']

In [23]:
# Display all the metrics of the model
metrics = best.get_metrics()
for i in metrics.keys():
    print(f"{i}: {metrics[i]:.3}")

StatementMeta(cba326d6-eba5-4cd2-97b0-716100838c47, 1, 40, Finished, Available, Finished)

log_loss: 0.325
average_precision_score_macro: 0.824
recall_score_weighted: 0.917
precision_score_micro: 0.917
AUC_weighted: 0.946
f1_score_weighted: 0.912
recall_score_micro: 0.917
recall_score_macro: 0.739
matthews_correlation: 0.54
weighted_accuracy: 0.961
precision_score_weighted: 0.91
norm_macro_recall: 0.478
AUC_micro: 0.981
precision_score_macro: 0.805
average_precision_score_weighted: 0.955
f1_score_micro: 0.917
f1_score_macro: 0.766
balanced_accuracy: 0.739
accuracy: 0.917
AUC_macro: 0.946
average_precision_score_micro: 0.981
accuracy_table: aml
confusion_matrix: aml
